# PA2 of CSE 251B, WI 21

## (a) Load training and testing data then create a validation split from the training data.

In [3]:
from neuralnet import *
# Load the configuration.
config = load_config("./")

# Create the model
model  = Neuralnetwork(config)

# Load the data
x_train, y_train = load_data(path="./", mode="train")
x_test,  y_test  = load_data(path="./", mode="t10k")

## (b)